In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
# from IPython.display import clear_output
import os
import random
from tqdm.auto import tqdm
from timeit import default_timer as timer

import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.models as KM
from tensorflow.keras.initializers import GlorotNormal

import sys
sys.path.append('/content/drive/My Drive/kaggle/contrail_detector/')
import models
import utils
from configs import wnet_rcnn_config, Path
print("TensorFlow version:", tf.__version__)

Mounted at /content/drive
TensorFlow version: 2.13.0


In [2]:
!pip install wandb
import wandb
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=71aca430126f478d887b40feac18d92f114b313faf15c3546e848f60e73b4e5a
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
train_metadata = pd.read_json("/content/drive/MyDrive/kaggle/contrail_detector/data/train_metadata.json", dtype={'record_id': 'str'})
valid_metadata = pd.read_json("/content/drive/MyDrive/kaggle/contrail_detector/data/valid_metadata.json", dtype={'record_id': 'str'})

In [4]:
train_metadata.head()

,record_id,projection_wkt,row_min,row_size,col_min,col_size,timestamp,contrail_exists
0,9075952105055778186,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",-1811231.625,-1901.528076,834079.00000,2024.375977,2019-08-28 23:20:00,0
1,8380074962411412067,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4696927.500,-2058.764893,592684.56250,1868.726685,2020-02-06 05:20:00,0
2,6808716999707654664,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4356032.000,-1996.296997,367297.18750,1928.051147,2020-03-04 21:10:00,1
3,8019228729201913883,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",4691404.500,-2039.323608,516009.65625,1887.738647,2019-10-17 10:00:00,1
4,1264966833918307888,"PROJCS[""unknown"",GEOGCS[""unknown"",DATUM[""WGS_1...",1187218.375,-1949.495972,243261.87500,1955.620239,2019-10-14 00:00:00,0


In [5]:
def load_data(record_id, path):
    # make path objects
    record_id = record_id.numpy().decode('utf-8')
    path = path.numpy().decode('utf-8')
    image_path = path + "image/" + str(record_id) + ".npy"
    mask_path = path + "mask/" + str(record_id) + ".npy"
    rpn_match_path = path + "rpn_matches/" + str(record_id) + ".npy"
    rpn_deltas_path = path + "rpn_deltas/" + str(record_id) + ".npy"
    gt_class_ids_path = path + "gt_class_ids/" + str(record_id) + ".npy"
    gt_boxes_path = path + "gt_bboxes/" + str(record_id) + ".npy"
    # load the images and masks for the correspoding paths
    normalized_anchors = np.load("/content/drive/MyDrive/kaggle/contrail_detector/data/normalized_anchors.npy").astype(np.float32)
    image = np.load(image_path)[...,wnet_rcnn_config.n_times_before, :].astype(np.float32)
    mask = np.load(mask_path).astype(np.float32)
    rpn_match = np.load(rpn_match_path).astype(np.int32)
    rpn_match= tf.expand_dims(rpn_match, axis=-1)
    rpn_deltas = np.load(rpn_deltas_path).astype(np.float32)
    gt_class_ids = np.load(gt_class_ids_path).astype(np.int32)
    gt_boxes = np.load(gt_boxes_path).astype(np.float32)

    return image, normalized_anchors, gt_class_ids, gt_boxes, mask, rpn_match, rpn_deltas

"""
def augment_data(image, mask):
    # random flip
    image_mask_concat = tf.concat([image, mask], axis = -1)
    image_mask_concat = tf.image.random_flip_left_right(image_mask_concat)
    image_mask_concat = tf.image.random_flip_up_down(image_mask_concat)
    # random crop
    cropped = tf.image.random_crop(image_mask_concat, size=[(Config.height*3)//4,(Config.width*3)//4, 4])
    image_mask_concat = tf.image.resize(cropped, [Config.height, Config.width])

    image = image_mask_concat[..., :-1]
    mask = image_mask_concat[..., -1]
    mask = tf.expand_dims(mask, axis=-1)
    mask = tf.cast(mask > 0.5, tf.float32)
    return image, mask
"""

id_to_data = lambda record_id, path : tf.py_function(func = load_data, inp = [record_id, path], Tout = (tf.float32, tf.float32, tf.int32, tf.float32, tf.float32, tf.int32, tf.float32))
train_id = train_metadata.record_id.to_list()
valid_id = valid_metadata.record_id.to_list()

train_path = len(train_id)*[Path.train]
valid_path = len(valid_id)*[Path.valid]

train_id_dataset = tf.data.Dataset.from_tensor_slices(train_id)
train_path_dataset = tf.data.Dataset.from_tensor_slices(train_path)

valid_id_dataset = tf.data.Dataset.from_tensor_slices(valid_id)
valid_path_dataset = tf.data.Dataset.from_tensor_slices(valid_path)

train_dataset = tf.data.Dataset.zip((train_id_dataset, train_path_dataset)).map(id_to_data).shuffle(wnet_rcnn_config.buffer_size).batch(wnet_rcnn_config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
valid_dataset = tf.data.Dataset.zip((valid_id_dataset, valid_path_dataset)).map(id_to_data).shuffle(wnet_rcnn_config.buffer_size).batch(wnet_rcnn_config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)




In [6]:
#  Loss Functions

def batch_pack_graph(x, counts, n_rows):
    """
    Combine batches of various sizes into a single tensor.

    Args:
        x (tf.Tensor): Input tensor.
        counts (tf.Tensor): Number of valid entries for each batch.
        n_rows (int): Number of rows in the tensor.

    Returns:
        tf.Tensor: Combined tensor.
    """
    outputs = []
    for i in range(n_rows):
        outputs.append(x[i, :counts[i]])
    return tf.concat(outputs, axis=0)

def smooth_l1_loss(y_true, y_pred):
    """
    Implement the Smooth L1 loss.

    Args:
        y_true (tf.Tensor): Ground truth values.
        y_pred (tf.Tensor): Predicted values.

    Returns:
        tf.Tensor: Loss value.
    """
    diff = K.abs(y_true - y_pred)
    less_than_one = K.cast(K.less(diff, 1.0), "float32")
    loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
    return loss

def rpn_class_loss_graph(rpn_matches, rpn_class_logits):
    """
    RPN anchor classifier loss.

    Args:
        rpn_matches (tf.Tensor): [n_batches, n_anchors, 1]. Anchor match type. 1=positive, -1=negative, 0=neutral anchor.
        rpn_class_logits (tf.Tensor): [n_batches, n_anchors, 2]. Logits for RPN classifier.

    Returns:
        tf.Tensor: Loss value.
    """
    # Anchor class. Convert anchor match type to class type (1=positive, 0=negative, ignores neutral)
    rpn_matches = tf.squeeze(rpn_matches, -1)
    anchor_class = K.cast(K.equal(rpn_matches, 1), tf.int32)
    indices = tf.where(K.not_equal(rpn_matches, 0))
    rpn_class_logits = tf.gather_nd(rpn_class_logits, indices)
    anchor_class = tf.gather_nd(anchor_class, indices)
    # Use categorical cross-entropy on the positive and negative anchors
    loss = K.sparse_categorical_crossentropy(target=anchor_class,
                                             output=rpn_class_logits,
                                             from_logits=True)
    loss = K.switch(tf.size(loss) > 0, K.mean(loss), tf.constant(0.0))
    return loss



def rpn_bbox_loss_graph(config, target_rpn_deltas, rpn_matches, rpn_deltas):
    """
    Compute the RPN bounding box loss graph.

    Args:
        config (Config): Configuration object with various hyperparameters.
        target_rpn_deltas (tf.Tensor): [n_batches, max_positive_anchors, (dy, dx, log(dh), log(dw))].
            Ground truth bounding box deltas.
        rpn_matches (tf.Tensor): [n_batches, n_anchors, 1]. Match values (1=positive anchor, -1=negative, 0=neutral).
        rpn_deltas (tf.Tensor): [n_batches, n_anchors, (dy, dx, log(dh), log(dw))].
            Predicted bounding box deltas.

    Returns:
        tf.Tensor: RPN bounding box loss.
    """
    # Squeeze to remove last dimension
    rpn_matches = K.squeeze(rpn_matches, -1)
    # Get the positive anchors
    indices = tf.where(K.equal(rpn_matches, 1))
    # Get predicted bounding box deltas for the positive anchors
    rpn_deltas = tf.gather_nd(rpn_deltas, indices)
    # Count the number of positive anchors
    batch_counts = K.sum(K.cast(K.equal(rpn_matches, 1), tf.int32), axis=1)
    # Pack the ground truth bounding boxes according to batch counts
    target_rpn_deltas = utils.batch_pack_graph(target_rpn_deltas, batch_counts, config.batch_size)
    # Compute the Smooth L1 loss
    loss = smooth_l1_loss(target_rpn_deltas, rpn_deltas)
    # If no positive anchors, return 0
    loss = K.switch(tf.size(loss) > 0, K.mean(loss), tf.constant(0.0))
    return loss

def rcnn_class_loss_graph(target_class_ids, rcnn_class_logits):
    """
    Compute the RCNN class loss graph.

    Args:
        target_class_ids (tf.Tensor): [n_batches, n_rois]. Ground truth class IDs.
        pred_class_logits (tf.Tensor): [n_batches, n_rois, n_classes]. Predicted class logits.

    Returns:
        tf.Tensor: RCNN class loss.
    """
    # Cast target class ids to int64 for compatibility with tf operations
    target_class_ids = tf.cast(target_class_ids, tf.int64)
    # Compute the sparse softmax cross entropy loss
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=target_class_ids, logits=rcnn_class_logits)
    # If there are no target class IDs, return 0
    loss = K.switch(tf.size(loss) > 0, K.mean(loss), tf.constant(0.0))
    return loss

def rcnn_bbox_loss_graph(target_rcnn_deltas, target_class_ids, rcnn_deltas):
    """
    Compute the RCNN bounding box loss graph.

    Args:
        target_rcnn_deltas (tf.Tensor): [n_batches, n_rois, (dy, dx, log(dh), log(dw))].
            Ground truth bounding box deltas.
        target_class_ids (tf.Tensor): [n_batches, n_rois]. Ground truth class IDs.
        rcnn_deltas (tf.Tensor): [n_batches, n_rois, n_classes, (dy, dx, log(dh), log(dw))].
            Predicted bounding box deltas.

    Returns:
        tf.Tensor: RCNN bounding box loss.
    """
    # Reshape inputs
    target_class_ids = K.reshape(target_class_ids, (-1,))
    target_rcnn_deltas = K.reshape(target_rcnn_deltas, (-1, 4))
    rcnn_deltas = K.reshape(rcnn_deltas, (-1, K.int_shape(rcnn_deltas)[2], 4))
    # Get positive ROI indices and corresponding class ids
    positive_roi_ix = tf.where(target_class_ids > 0)[:, 0]
    positive_roi_class_ids = tf.cast(tf.gather(target_class_ids, positive_roi_ix), tf.int64)
    indices = tf.stack([positive_roi_ix, positive_roi_class_ids], axis=1)
    # Get corresponding target and predicted bounding boxes
    target_rcnn_deltas = tf.gather(target_rcnn_deltas, positive_roi_ix)
    rcnn_deltas = tf.gather_nd(rcnn_deltas, indices)
    # Compute the Smooth L1 loss
    loss = K.switch(tf.size(target_rcnn_deltas) > 0, smooth_l1_loss(y_true=target_rcnn_deltas, y_pred=rcnn_deltas), tf.constant(0.0))
    loss = K.mean(loss)
    return loss

def combined_loss(output_dict, rpn_matches, rpn_deltas, config) :
  rpn_class_loss = KL.Lambda(lambda x: rpn_class_loss_graph(*x), name="rpn_class_loss")(
      [rpn_matches, output_dict['rpn_class_logits']])
  rpn_bbox_loss = KL.Lambda(lambda x: rpn_bbox_loss_graph(config, *x), name="rpn_bbox_loss")(
      [rpn_deltas, rpn_matches, output_dict['normalized_rpn_deltas']])
  class_loss = KL.Lambda(lambda x: rcnn_class_loss_graph(*x), name="rcnn_class_loss")(
      [output_dict['target_class_ids'], output_dict['rcnn_class_logits']])
  bbox_loss = KL.Lambda(lambda x: rcnn_bbox_loss_graph(*x), name="rcnn_bbox_loss")(
      [output_dict['normalized_target_deltas'], output_dict['target_class_ids'], output_dict['normalized_rcnn_deltas']])

  # Summing up the individual loss terms
  total_loss = KL.Lambda(lambda x: tf.add_n(x), name="total_loss")(
    [rpn_class_loss, rpn_bbox_loss, class_loss, bbox_loss])

  return total_loss

In [7]:
def dice_score(y_true, y_pred, thr=0.5, epsilon=1e-6):
    # Thresholding predictions
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_pred = tf.cast(y_pred > thr, tf.float32)

    # Flattening tensors
    y_true = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

    # Computing intersections and unions
    intersections = tf.reduce_sum(y_true * y_pred, axis=1)
    unions = tf.reduce_sum(y_true, axis=1) + tf.reduce_sum(y_pred, axis=1)

    # Computing Dice scores
    dices = (2. * intersections + epsilon) / (unions + epsilon)

    return tf.reduce_mean(dices)

# Define Dice Loss
# we do not binarize the prediction for dice 'loss' calculation

In [8]:
@tf.function
def train_step(inputs, model, loss_fn, optimizer, config):
  with tf.GradientTape() as tape:
    model.mode = 'training'
    output_dict = model(*inputs[:4])
    loss_value = loss_fn(output_dict, *inputs[5:7], config)
  gradients = tape.gradient(loss_value, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss_value

@tf.function
def valid_step1(inputs, model, loss_fn, metric_fn, config):
  model.mode = 'training'
  output_dict = model(*inputs[:4])
  loss_value = loss_fn(output_dict, *inputs[5:7], config)
  return loss_value

def valid_step2(inputs, model, loss_fn, metric_fn, config) :
  model.mode = 'inference'
  output_dict = model(*inputs[:4])
  metric_value= metric_fn(output_dict['gt_attention_masks'], output_dict['attention_masks'])
  return metric_value

def train_model(train_dataset, valid_dataset, model, loss_fn, metric_fn, optimizer, epochs, checkpoint_directory, log_directory, resume = None, max_to_keep=3, config = wnet_rcnn_config):

  # printout which device the model is on
  # print(model.layers[0].weights[0].device)

  train_losses = []
  valid_losses = []
  valid_metrics = []

  # Create a checkpoint object and checkpoint manager
  checkpoint = tf.train.Checkpoint(step=tf.Variable(0),
                                   optimizer=optimizer,
                                   model=model)
  manager = tf.train.CheckpointManager(checkpoint,
                                       checkpoint_directory,
                                       max_to_keep)

  # restore the last checkpoint if there is one
  checkpoint.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  train_summary_writer = tf.summary.create_file_writer(log_directory)
  valid_summary_writer = tf.summary.create_file_writer(log_directory)

  # Initialize W&B run
  wandb.init(project='contrail_detector', name='wnet_rcnn', resume = resume)


  # training loop starts
  for epoch in range(epochs):
    start_time = timer()
    checkpoint.step.assign_add(1)
    print(f"\n Epoch {checkpoint.step.numpy()} : \n")

    # initialize losses and metrics
    train_total_loss = 0
    valid_total_loss = 0
    valid_total_metric = 0
    train_n_batches = 0
    valid_n_batches = 0

    # train loop
    for step, inputs_batch in enumerate(tqdm(train_dataset, desc='Training', position=0)):
      loss_value = train_step(inputs_batch, model, loss_fn, optimizer, config)

      train_total_loss += loss_value
      train_n_batches += 1

      if step % 1 == 0: # print every 1 batches
        print(f"batch {step}, train_loss: {loss_value:.4f}")

    # valid loop
    for step, inputs_batch in enumerate(tqdm(valid_dataset, desc='Validation', position=0)):
      loss_value = valid_step1(inputs_batch, model, loss_fn, metric_fn, config)
      metric_value= valid_step2(inputs_batch, model, loss_fn, metric_fn, config)
      valid_total_loss += loss_value
      valid_total_metric += metric_value
      valid_n_batches += 1

    train_epoch_loss = train_total_loss / train_n_batches
    valid_epoch_loss = valid_total_loss / valid_n_batches
    valid_epoch_metric = valid_total_metric / valid_n_batches

    train_losses.append(train_epoch_loss)
    valid_losses.append(valid_epoch_loss)
    valid_metrics.append(valid_epoch_metric)

    # Log training metrics for the epoch
    with train_summary_writer.as_default():
        tf.summary.scalar('train_loss', train_epoch_loss, step=checkpoint.step.numpy())

    # Log validation metrics for the epoch
    with valid_summary_writer.as_default():
        tf.summary.scalar('valid_loss', valid_epoch_loss, step=checkpoint.step.numpy())
        tf.summary.scalar('valid_metric', valid_epoch_metric, step=checkpoint.step.numpy())

    #save checkpoint
    save_path = manager.save()

    # Log metrics to W&B
    wandb.log({
              'train_loss': train_epoch_loss.numpy(),
              'valid_loss': valid_epoch_loss.numpy(),
              'valid_metric': valid_epoch_metric.numpy(),
              'epoch': checkpoint.step.numpy()
          })
    #save checkpoint
    save_path = manager.save()

    # Save checkpoint to W&B as an artifact
    artifact = wandb.Artifact(f'wnet_rcnn-epoch-{checkpoint.step.numpy()}-checkpoint', type='model')
    # Add the index file
    artifact.add_file(save_path + ".index")

    # Add the data file
    artifact.add_file(save_path + ".data-00000-of-00001")

    wandb.log_artifact(artifact)
    end_time = timer()

    # print out results
    print("\n")
    print("  (Results)")
    print(f"  train_loss: {train_epoch_loss:.4f}")
    print(f"  valid_loss: {valid_epoch_loss:.4f}")
    print(f"  valid_metric: {valid_epoch_metric:.4f}")
    print("\n")
    print("  (Time)")
    print("  start_time : ", start_time)
    print("  end_time : ", end_time)
    print("-----------------------------------------------------------------")

  # Finish the W&B run
  wandb.finish()

  return train_losses, valid_losses, valid_metrics

In [9]:
model = models.rcnn('training', wnet_rcnn_config, architecture="resnet50")
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(wnet_rcnn_config.initial_learning_rate,
                                                        decay_steps=wnet_rcnn_config.decay_steps,
                                                        alpha=wnet_rcnn_config.alpha)
optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule, clipnorm = wnet_rcnn_config.gradient_clipnorm)
loss_fn = lambda output_dict,  rpn_matches, rpn_deltas, config : combined_loss(output_dict, rpn_matches, rpn_deltas, config)
metric_fn = lambda y_true, y_pred : dice_score(y_true, y_pred, thr=0.5, epsilon=1e-6)

102967424/102967424 [==============================] - 6s 0us/step


In [10]:
checkpoint_directory = "/content/drive/MyDrive/kaggle/contrail_detector/checkpoint/wnet_rcnn"
log_directory = "/content/drive/MyDrive/kaggle/contrail_detector/log/wnet_rcnn"
train_losses, valid_losses, valid_metrics = train_model(train_dataset, valid_dataset, model,
                                                                                loss_fn, metric_fn, optimizer,
                                                                                epochs=wnet_rcnn_config.n_epochs,
                                                                                checkpoint_directory = checkpoint_directory,
                                                                                log_directory = log_directory)

Initializing from scratch.


wandb: Currently logged in as: jun23ird. Use `wandb login --relogin` to force relogin



 Epoch 1 : 



Training:   0%|          | 0/20 [00:00<?, ?it/s]

batch 0, train_loss: 19350.0332
batch 1, train_loss: 22981.6992
batch 2, train_loss: 30306.5078
batch 3, train_loss: 22725.5332
batch 4, train_loss: 19096.4590
batch 5, train_loss: 16931.4551
batch 6, train_loss: 20403.3320
batch 7, train_loss: 23966.3105
batch 8, train_loss: 16872.0898
batch 9, train_loss: 23126.0195
batch 10, train_loss: 17573.7109
batch 11, train_loss: 15819.3633
batch 12, train_loss: 22979.7578
batch 13, train_loss: 18666.5957
batch 14, train_loss: 18657.2500
batch 15, train_loss: 22807.0098
batch 16, train_loss: 18865.3809
batch 17, train_loss: 14116.2285
batch 18, train_loss: 19241.9277
batch 19, train_loss: 21549.0859


...

Training:   0%|          | 0/20 [00:00<?, ?it/s]

batch 0, train_loss: 20872.8281
batch 1, train_loss: 22689.3086
batch 2, train_loss: 24627.9180
batch 3, train_loss: 26075.8281
batch 4, train_loss: 17411.6230
batch 5, train_loss: 18705.2188
batch 6, train_loss: 21821.1055
batch 7, train_loss: 17875.8242
batch 8, train_loss: 22123.7109
batch 9, train_loss: 19945.3086
batch 10, train_loss: 16352.2998
batch 11, train_loss: 18512.5391
batch 12, train_loss: 13032.7441
batch 13, train_loss: 21991.0762
batch 14, train_loss: 19223.5684
batch 15, train_loss: 22074.8281
batch 16, train_loss: 18982.7871
batch 17, train_loss: 16608.1152
batch 18, train_loss: 17609.2754
batch 19, train_loss: 21933.7773


Validation:   0%|          | 0/5 [00:00<?, ?it/s]



  (Results)
  train_loss: 19923.4863
  valid_loss: 19122.8633
  valid_metric: 0.9700


  (Time)
  start_time :  4620.750758689
  end_time :  4656.374765369
-----------------------------------------------------------------


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,▆▃▄█▆▅▅▅▄▂▇▄▅▆▅▆▅▄▄▅▃▆▃▄▅▇▆▅▁▆▅▅▃▄▄▃▆▆▁▃
valid_loss,▅▅▄▄▄▅▅▆▃▄▄▄▅▄▄▆▅▄▃▄▄▅▄▅▁▂▆▄▅▃█▄▃▂▄▃▂▁▃▂
valid_metric,█▁▁▁█▁█▁▁▁▁█▁▁████▁▁▁█▁███▁▁▁▁▁█▁▁▁▁█▁██
epoch,60
train_loss,19923.48633
valid_loss,19122.86328
valid_metric,0.97
